# 질문 생성을 통한 문서 증강 RAG

이 노트북은 질문 생성을 통한 문서 증강을 사용하여 향상된 RAG 접근 방식을 구현합니다. 각 텍스트 청크에 대해 관련 질문을 생성함으로써 검색 프로세스를 개선하여 언어 모델로부터 더 나은 응답을 이끌어냅니다.
문서 증강(Document Augmentation)은 기존 문서의 내용을 바탕으로 추가적인 정보(여기서는 질문)를 생성하여 검색 효율성과 정확성을 높이는 기법입니다. 각 텍스트 청크에 대해 "이 청크는 어떤 질문에 답할 수 있는가?"와 같은 질문들을 미리 생성해두고, 사용자의 실제 질의가 들어왔을 때, 사용자의 질의와 가장 유사한 미리 생성된 질문을 찾아 해당 질문과 연결된 청크를 검색 결과로 제공합니다. 이는 사용자의 다양한 질의 형태에 더 유연하게 대응할 수 있도록 돕습니다.

이 구현에서는 다음 단계를 따릅니다:

1. **데이터 수집**: PDF 파일에서 텍스트를 추출합니다.
2. **청킹**: 텍스트를 관리 가능한 청크로 분할합니다.
3. **질문 생성**: 각 청크에 대해 관련 질문을 생성합니다.
4. **임베딩 생성**: 청크와 생성된 질문 모두에 대한 임베딩을 생성합니다.
5. **벡터 저장소 생성**: NumPy를 사용하여 간단한 벡터 저장소를 구축합니다.
6. **시맨틱 검색**: 사용자 질의에 대한 관련 청크 및 질문을 검색합니다.
7. **응답 생성**: 검색된 내용을 기반으로 답변을 생성합니다.
8. **평가**: 생성된 응답의 품질을 평가합니다.

## 환경 설정
필요한 라이브러리를 가져오는 것으로 시작합니다.

In [ ]:
import fitz # PyMuPDF
import os
import numpy as np
import json
from openai import OpenAI
import re # 정규 표현식
from tqdm import tqdm # 진행률 표시

## PDF 파일에서 텍스트 추출
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 여기서는 PyMuPDF 라이브러리를 사용하여 PDF 파일에서 텍스트를 추출합니다.

In [2]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출하고 처음 `num_chars`개의 문자를 출력합니다.

    Args:
    pdf_path (str): PDF 파일 경로.

    Returns:
    str: PDF에서 추출된 텍스트.
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 텍스트를 저장할 빈 문자열 초기화

    # PDF의 각 페이지 반복
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # 페이지 가져오기
        text = page.get_text("text")  # 페이지에서 텍스트 추출
        all_text += text  # 추출된 텍스트를 all_text 문자열에 추가

    return all_text  # 추출된 텍스트 반환

## 추출된 텍스트 청킹
추출된 텍스트가 있으면 검색 정확도를 높이기 위해 더 작고 중첩되는 청크로 나눕니다.

In [3]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n개의 문자로 된 세그먼트로 나누고 중첩을 허용합니다.

    Args:
    text (str): 청킹할 텍스트.
    n (int): 각 청크의 문자 수.
    overlap (int): 청크 간 중첩되는 문자 수.

    Returns:
    List[str]: 텍스트 청크 목록.
    """
    chunks = []  # 청크를 저장할 빈 리스트 초기화
    
    # (n - overlap) 크기의 단계로 텍스트 반복
    for i in range(0, len(text), n - overlap):
        # 인덱스 i부터 i + n까지의 텍스트 청크를 청크 목록에 추가
        chunks.append(text[i:i + n])

    return chunks  # 텍스트 청크 목록 반환

## OpenAI API 클라이언트 설정
임베딩과 응답을 생성하기 위해 OpenAI 클라이언트를 초기화합니다.

In [ ]:
# 기본 URL과 API 키로 OpenAI 클라이언트 초기화
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 검색
)

## 텍스트 청크에 대한 질문 생성
이것이 간단한 RAG에 비해 핵심적인 개선 사항입니다. 각 텍스트 청크로 답변할 수 있는 질문을 생성합니다.
LLM을 사용하여 각 청크의 내용을 기반으로 여러 개의 가상 질문을 만듭니다. 이 질문들은 해당 청크의 핵심 정보를 담고 있어야 합니다.

In [5]:
def generate_questions(text_chunk, num_questions=5, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    주어진 텍스트 청크에서 답변할 수 있는 관련 질문을 생성합니다.

    Args:
    text_chunk (str): 질문을 생성할 텍스트 청크.
    num_questions (int): 생성할 질문 수.
    model (str): 질문 생성에 사용할 모델.

    Returns:
    List[str]: 생성된 질문 목록.
    """
    # AI의 행동을 안내하는 시스템 프롬프트 정의
    system_prompt = "당신은 텍스트에서 관련 질문을 생성하는 전문가입니다. 제공된 텍스트만을 사용하여 답변할 수 있는 간결한 질문을 만드십시오. 핵심 정보와 개념에 집중하십시오."
    
    # 텍스트 청크와 생성할 질문 수를 포함하는 사용자 프롬프트 정의
    user_prompt = f"""
    다음 텍스트를 기반으로, 이 텍스트만을 사용하여 답변할 수 있는 {num_questions}개의 서로 다른 질문을 생성하십시오:

    {text_chunk}
    
    추가 텍스트 없이 번호 매기기 목록 형식으로 질문만 응답하십시오.
    """
    
    # OpenAI API를 사용하여 질문 생성
    response = client.chat.completions.create(
        model=model,
        temperature=0.7, # 약간의 창의성을 허용하기 위해 온도를 0.7로 설정
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # 응답에서 질문 추출 및 정리
    questions_text = response.choices[0].message.content.strip()
    questions = []
    
    # 정규 표현식 패턴 일치를 사용하여 질문 추출
    for line in questions_text.split('\n'):
        # 번호 매기기 제거 및 공백 정리
        cleaned_line = re.sub(r'^\d+\.\s*', '', line.strip()) # 숫자로 시작하고 점과 공백이 오는 패턴 제거
        if cleaned_line and cleaned_line.endswith('?'): # 정리된 줄이 비어있지 않고 물음표로 끝나는지 확인
            questions.append(cleaned_line)
    
    return questions

## 텍스트에 대한 임베딩 생성
텍스트 청크와 생성된 질문 모두에 대한 임베딩을 생성합니다.

In [6]:
def create_embeddings(text, model="BAAI/bge-en-icl"):
    """
    지정된 OpenAI 모델을 사용하여 주어진 텍스트에 대한 임베딩을 생성합니다.

    Args:
    text (str): 임베딩을 생성할 입력 텍스트.
    model (str): 임베딩 생성에 사용할 모델.

    Returns:
    dict: 임베딩을 포함하는 OpenAI API의 응답.
    """
    # 지정된 모델을 사용하여 입력 텍스트에 대한 임베딩 생성
    response = client.embeddings.create(
        model=model,
        input=text
    )

    return response  # 임베딩을 포함하는 응답 반환

## 간단한 벡터 저장소 구축
NumPy를 사용하여 간단한 벡터 저장소를 구현합니다.
이 벡터 저장소에는 원본 텍스트 청크와 해당 청크에서 생성된 가상 질문들이 모두 임베딩된 형태로 저장됩니다. 각 항목은 원본 텍스트, 임베딩 벡터, 그리고 메타데이터(예: 해당 항목이 '청크'인지 '질문'인지, 질문이라면 어떤 청크에서 생성되었는지 등)를 포함합니다.

In [7]:
class SimpleVectorStore:
    """
    NumPy를 사용한 간단한 벡터 저장소 구현.
    """
    def __init__(self):
        """
        벡터 저장소를 초기화합니다.
        """
        self.vectors = [] # 임베딩 벡터 저장
        self.texts = []   # 원본 텍스트 (청크 또는 질문) 저장
        self.metadata = [] # 메타데이터 저장
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 항목을 추가합니다.

        Args:
        text (str): 원본 텍스트.
        embedding (List[float]): 임베딩 벡터.
        metadata (dict, optional): 추가 메타데이터.
        """
        self.vectors.append(np.array(embedding))
        self.texts.append(text)
        self.metadata.append(metadata or {})
    
    def similarity_search(self, query_embedding, k=5):
        """
        질의 임베딩과 가장 유사한 항목을 찾습니다.

        Args:
        query_embedding (List[float]): 질의 임베딩 벡터.
        k (int): 반환할 결과 수.

        Returns:
        List[Dict]: 텍스트 및 메타데이터와 함께 상위 k개의 가장 유사한 항목.
        """
        if not self.vectors:
            return []
        
        # 질의 임베딩을 numpy 배열로 변환
        query_vector = np.array(query_embedding)
        
        # 코사인 유사도를 사용하여 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))
        
        # 유사도 기준으로 정렬 (내림차순)
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개 결과 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score
            })
        
        return results

## 질문 증강을 통한 문서 처리
이제 모든 것을 통합하여 문서를 처리하고, 질문을 생성하며, 증강된 벡터 저장소를 구축합니다.

In [8]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200, questions_per_chunk=5):
    """
    질문 증강을 통해 문서를 처리합니다.

    Args:
    pdf_path (str): PDF 파일 경로.
    chunk_size (int): 각 텍스트 청크의 문자 크기.
    chunk_overlap (int): 청크 간 문자 중첩.
    questions_per_chunk (int): 청크당 생성할 질문 수.

    Returns:
    Tuple[List[str], SimpleVectorStore]: 텍스트 청크 및 벡터 저장소.
    """
    print("PDF에서 텍스트 추출 중...")
    extracted_text = extract_text_from_pdf(pdf_path)
    
    print("텍스트 청킹 중...")
    text_chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"생성된 텍스트 청크 수: {len(text_chunks)}")
    
    vector_store = SimpleVectorStore()
    
    print("청크 처리 및 질문 생성 중...")
    for i, chunk in enumerate(tqdm(text_chunks, desc="청크 처리 중")):
        # 청크 자체에 대한 임베딩 생성
        chunk_embedding_response = create_embeddings(chunk)
        chunk_embedding = chunk_embedding_response.data[0].embedding
        
        # 벡터 저장소에 청크 추가
        vector_store.add_item(
            text=chunk,
            embedding=chunk_embedding,
            metadata={"type": "chunk", "index": i} # 메타데이터에 'chunk' 타입과 인덱스 저장
        )
        
        # 이 청크에 대한 질문 생성
        questions = generate_questions(chunk, num_questions=questions_per_chunk)
        
        # 각 질문에 대한 임베딩을 생성하고 벡터 저장소에 추가
        for j, question in enumerate(questions):
            question_embedding_response = create_embeddings(question)
            question_embedding = question_embedding_response.data[0].embedding
            
            # 벡터 저장소에 질문 추가
            vector_store.add_item(
                text=question,
                embedding=question_embedding,
                metadata={"type": "question", "chunk_index": i, "original_chunk": chunk} # 메타데이터에 'question' 타입, 원본 청크 인덱스, 원본 청크 내용 저장
            )
    
    return text_chunks, vector_store

## 문서 추출 및 처리

In [9]:
# PDF 파일 경로 정의
pdf_path = "data/AI_Information.pdf"

# 문서 처리 (텍스트 추출, 청크 생성, 질문 생성, 벡터 저장소 구축)
text_chunks, vector_store = process_document(
    pdf_path, 
    chunk_size=1000, 
    chunk_overlap=200, 
    questions_per_chunk=3 # 각 청크당 3개의 질문 생성
)

print(f"벡터 저장소에는 {len(vector_store.texts)}개의 항목이 포함되어 있습니다.")

Extracting text from PDF...
Chunking text...
Created 42 text chunks
Processing chunks and generating questions...


Processing Chunks: 100%|██████████| 42/42 [01:30<00:00,  2.15s/it]

Vector store contains 165 items


## 시맨틱 검색 수행
간단한 RAG 구현과 유사하지만 증강된 벡터 저장소에 맞게 조정된 시맨틱 검색 기능을 구현합니다.
사용자 질의가 들어오면, 벡터 저장소에 저장된 모든 항목(원본 청크 및 생성된 질문)의 임베딩과 사용자 질의 임베딩 간의 유사도를 계산하여 가장 관련성 높은 항목들을 찾습니다.

In [10]:
def semantic_search(query, vector_store, k=5):
    """
    질의와 벡터 저장소를 사용하여 시맨틱 검색을 수행합니다.

    Args:
    query (str): 검색 질의.
    vector_store (SimpleVectorStore): 검색할 벡터 저장소.
    k (int): 반환할 결과 수.

    Returns:
    List[Dict]: 상위 k개의 가장 관련성 높은 항목.
    """
    # 질의에 대한 임베딩 생성
    query_embedding_response = create_embeddings(query)
    query_embedding = query_embedding_response.data[0].embedding
    
    # 벡터 저장소 검색
    results = vector_store.similarity_search(query_embedding, k=k)
    
    return results

## 증강된 벡터 저장소에 대한 질의 실행

In [11]:
# JSON 파일에서 검증 데이터 로드
with open('data/val.json') as f:
    data = json.load(f)

# 검증 데이터에서 첫 번째 질의 추출
query = data[0]['question']

# 관련 콘텐츠를 찾기 위해 시맨틱 검색 수행
search_results = semantic_search(query, vector_store, k=5)

print("질의:", query)
print("\n검색 결과:")

# 결과를 유형별로 정리
chunk_results = []
question_results = []

for result in search_results:
    if result["metadata"]["type"] == "chunk":
        chunk_results.append(result)
    else:
        question_results.append(result)

# 청크 결과 먼저 출력
print("\n관련 문서 청크:")
for i, result in enumerate(chunk_results):
    print(f"문맥 {i + 1} (유사도: {result['similarity']:.4f}):")
    print(result["text"][:300] + "...") # 처음 300자만 출력
    print("=====================================")

# 그런 다음 질문 일치 항목 출력
print("\n일치하는 질문:")
for i, result in enumerate(question_results):
    print(f"질문 {i + 1} (유사도: {result['similarity']:.4f}):")
    print(result["text"])
    chunk_idx = result["metadata"]["chunk_index"]
    print(f"청크 {chunk_idx}에서 생성됨")
    print("=====================================")

Query: What is 'Explainable AI' and why is it considered important?

Search Results:

Relevant Document Chunks:

Matched Questions:
Question 1 (similarity: 0.8629):
What is the main goal of Explainable AI (XAI)?
From chunk 10
Question 2 (similarity: 0.8488):
What is the primary goal of Explainable AI (XAI) techniques?
From chunk 37
Question 3 (similarity: 0.8414):
What is the focus of research on Explainable AI (XAI)?
From chunk 29
Question 4 (similarity: 0.7995):
Why are transparency and explainability essential for building trust in AI systems?
From chunk 36
Question 5 (similarity: 0.7841):
Why is transparency and explainability essential in building trust and accountability with AI systems?
From chunk 9


## 응답을 위한 컨텍스트 생성
이제 관련 청크와 질문의 정보를 결합하여 컨텍스트를 준비합니다.
검색 결과에는 원본 청크와 생성된 질문이 섞여 있을 수 있습니다. 답변 생성을 위해 LLM에 전달할 최종 컨텍스트를 만들 때는, (1) 직접 검색된 청크와 (2) 검색된 질문과 연결된 원본 청크들을 모두 포함시킵니다. 중복을 제거하여 고유한 청크들만 컨텍스트로 구성합니다.

In [12]:
def prepare_context(search_results):
    """
    응답 생성을 위해 검색 결과에서 통합된 컨텍스트를 준비합니다.

    Args:
    search_results (List[Dict]): 시맨틱 검색 결과.

    Returns:
    str: 결합된 컨텍스트 문자열.
    """
    # 결과에서 참조된 고유 청크 추출
    chunk_indices = set() # 중복 방지를 위해 set 사용
    context_chunks = []
    
    # 먼저 직접적인 청크 일치 항목 추가
    for result in search_results:
        if result["metadata"]["type"] == "chunk":
            chunk_indices.add(result["metadata"]["index"])
            context_chunks.append(f"청크 {result['metadata']['index']}:\n{result['text']}")
    
    # 그런 다음 질문으로 참조된 청크 추가
    for result in search_results:
        if result["metadata"]["type"] == "question":
            chunk_idx = result["metadata"]["chunk_index"]
            if chunk_idx not in chunk_indices: # 이미 추가된 청크는 건너뜀
                chunk_indices.add(chunk_idx)
                context_chunks.append(f"청크 {chunk_idx} (질문 '{result['text']}'에 의해 참조됨):\n{result['metadata']['original_chunk']}")
    
    # 모든 컨텍스트 청크 결합
    full_context = "\n\n".join(context_chunks)
    return full_context

## 검색된 청크를 기반으로 응답 생성


In [13]:
def generate_response(query, context, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    질의와 컨텍스트를 기반으로 응답을 생성합니다.

    Args:
    query (str): 사용자의 질문.
    context (str): 벡터 저장소에서 검색된 컨텍스트 정보.
    model (str): 응답 생성에 사용할 모델.

    Returns:
    str: 생성된 응답.
    """
    system_prompt = "당신은 주어진 문맥에 기반하여 엄격하게 답변하는 AI 어시스턴트입니다. 제공된 문맥에서 직접적으로 답변을 도출할 수 없는 경우, '답변하기에 충분한 정보가 없습니다.'라고 응답하십시오."
    
    user_prompt = f"""
        문맥:
        {context}

        질문: {query}

        위에 제공된 문맥만을 기반으로 질문에 답변하십시오. 간결하고 정확하게 답변하십시오.
    """
    
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return response.choices[0].message.content

## 응답 생성 및 표시

In [14]:
# 검색 결과에서 컨텍스트 준비
context = prepare_context(search_results)

# 응답 생성
response_text = generate_response(query, context)

print("\n질의:", query)
print("\n응답:")
print(response_text)


Query: What is 'Explainable AI' and why is it considered important?

Response:
Explainable AI (XAI) is a field that aims to make AI systems more transparent and understandable by providing insights into how AI models make decisions. This is essential for building trust and accountability in AI systems, as it enables users to assess their fairness and accuracy. XAI techniques are crucial for addressing potential harms, ensuring ethical behavior, and establishing clear guidelines and ethical frameworks for AI development and deployment.


## AI 응답 평가
AI 응답을 예상 답변과 비교하고 점수를 매깁니다.

In [15]:
def evaluate_response(query, response, reference_answer, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    AI 응답을 참조 답변과 비교하여 평가합니다.
    
    Args:
    query (str): 사용자의 질문.
    response (str): AI 생성 응답.
    reference_answer (str): 참조/이상적인 답변.
    model (str): 평가에 사용할 모델.
    
    Returns:
    str: 평가 피드백.
    """
    # 평가 시스템에 대한 시스템 프롬프트 정의
    evaluate_system_prompt = """당신은 AI 응답을 평가하는 지능형 평가 시스템입니다.
            
        AI 어시스턴트의 응답을 실제/참조 답변과 비교하여 다음을 기준으로 평가하십시오:
        1. 사실적 정확성 - 응답에 정확한 정보가 포함되어 있습니까?
        2. 완전성 - 참조 답변의 모든 중요한 측면을 다루고 있습니까?
        3. 관련성 - 질문을 직접적으로 다루고 있습니까?

        0에서 1까지의 점수를 부여하십시오:
        - 1.0: 내용과 의미에서 완벽하게 일치
        - 0.8: 사소한 누락/차이점이 있지만 매우 좋음
        - 0.6: 주요 요점을 다루지만 일부 세부 사항 누락
        - 0.4: 상당한 누락이 있는 부분적인 답변
        - 0.2: 최소한의 관련 정보
        - 0.0: 부정확하거나 관련 없음

        정당화와 함께 점수를 제공하십시오.
    """
            
    # 평가 프롬프트 생성
    evaluation_prompt = f"""
        사용자 질의: {query}

        AI 응답:
        {response}

        참조 답변:
        {reference_answer}

        AI 응답을 참조 답변과 비교하여 평가하십시오.
    """
    
    # 평가 생성
    eval_response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": evaluate_system_prompt},
            {"role": "user", "content": evaluation_prompt}
        ]
    )
    
    return eval_response.choices[0].message.content

## 평가 실행

In [16]:
# 검증 데이터에서 참조 답변 가져오기
reference_answer = data[0]['ideal_answer']

# 응답 평가
evaluation = evaluate_response(query, response_text, reference_answer)

print("\n평가:")
print(evaluation)


Evaluation:
Based on the evaluation criteria, I will assess the AI response as follows:

1. Factual correctness: The AI response contains accurate information about Explainable AI (XAI) and its importance. It correctly states that XAI aims to make AI systems more transparent and understandable, providing insights into how they make decisions.

2. Completeness: The AI response covers the main points of XAI, including its importance for building trust, accountability, and ensuring fairness in AI systems. However, it misses some details, such as the potential harms that XAI can address and the need for clear guidelines and ethical frameworks for AI development and deployment.

3. Relevance: The AI response directly addresses the question, providing a clear and concise explanation of XAI and its significance.

Based on the evaluation, I would assign a score of 0.8 to the AI response. The response is very good, with minor omissions and differences from the reference answer. It covers the m

## PDF 파일에서 텍스트 추출 및 청킹 (이 부분은 이전 셀들과 중복되어 보입니다. 필요시 삭제하거나 주석 처리할 수 있습니다.)
이제 PDF를 로드하고, 텍스트를 추출한 다음, 청크로 분할합니다.

In [17]:
# PDF 파일 경로 정의
pdf_path = "data/AI_Information.pdf"

# PDF 파일에서 텍스트 추출
extracted_text = extract_text_from_pdf(pdf_path)

# 추출된 텍스트를 1000자 세그먼트로 청킹하고 200자 중첩 허용
text_chunks = chunk_text(extracted_text, 1000, 200)

# 생성된 텍스트 청크 수 출력
print("텍스트 청크 수:", len(text_chunks))

# 첫 번째 텍스트 청크 출력
print("\n첫 번째 텍스트 청크:")
print(text_chunks[0])

Number of text chunks: 42

First text chunk:
Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing power and data availability 
have significantly accelerated the development and deployment of AI. 
Historical Context 
The idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. 
However, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop 
in 1956 is widely considered the birthplace of AI. Early AI research focused on problem-solving 
and 

## 텍스트 청크에 대한 임베딩 생성 (이 부분은 이전 셀들과 중복되어 보입니다.)
임베딩은 텍스트를 숫자 벡터로 변환하여 효율적인 유사도 검색을 가능하게 합니다.

In [18]:
def create_embeddings(text, model="BAAI/bge-en-icl"):
    """
    지정된 OpenAI 모델을 사용하여 주어진 텍스트에 대한 임베딩을 생성합니다.

    Args:
    text (str): 임베딩을 생성할 입력 텍스트.
    model (str): 임베딩 생성에 사용할 모델. 기본값은 "BAAI/bge-en-icl"입니다.

    Returns:
    dict: 임베딩을 포함하는 OpenAI API의 응답.
    """
    # 지정된 모델을 사용하여 입력 텍스트에 대한 임베딩 생성
    response = client.embeddings.create(
        model=model,
        input=text
    )

    return response  # 임베딩을 포함하는 응답 반환

# 텍스트 청크에 대한 임베딩 생성
response = create_embeddings(text_chunks) # 이 response 변수는 이전 질문 생성 로직에서 사용된 response와 다름에 유의

## 시맨틱 검색 수행 (이 부분은 이전 셀들과 중복되어 보입니다.)
사용자 질의에 가장 관련성 높은 텍스트 청크를 찾기 위해 코사인 유사도를 구현합니다.

In [19]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터.
    vec2 (np.ndarray): 두 번째 벡터.

    Returns:
    float: 두 벡터 간의 코사인 유사도.
    """
    # 두 벡터의 내적을 계산하고 각 벡터의 노름(크기)의 곱으로 나눕니다.
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [20]:
def semantic_search(query, text_chunks, embeddings, k=5):
    """
    주어진 질의와 임베딩을 사용하여 텍스트 청크에 대한 시맨틱 검색을 수행합니다.

    Args:
    query (str): 시맨틱 검색을 위한 질의.
    text_chunks (List[str]): 검색할 텍스트 청크 목록.
    embeddings (List[dict]): 텍스트 청크에 대한 임베딩 목록 (여기서는 create_embeddings 함수의 반환값 전체를 의미).
    k (int): 반환할 상위 관련 텍스트 청크 수. 기본값은 5입니다.

    Returns:
    List[str]: 질의를 기반으로 한 상위 k개의 가장 관련성 높은 텍스트 청크 목록.
    """
    # 질의에 대한 임베딩 생성
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []  # 유사도 점수를 저장할 리스트 초기화

    # 질의 임베딩과 각 텍스트 청크 임베딩 간의 유사도 점수 계산
    # embeddings.data 를 사용해야 함 (create_embeddings 함수의 반환값이 OpenAI API 응답 객체이므로)
    for i, chunk_embedding_obj in enumerate(embeddings.data):
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding_obj.embedding))
        similarity_scores.append((i, similarity_score))  # 인덱스와 유사도 점수 추가

    # 유사도 점수를 내림차순으로 정렬
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    # 상위 k개의 가장 유사한 텍스트 청크의 인덱스 가져오기
    top_indices = [index for index, _ in similarity_scores[:k]]
    # 상위 k개의 가장 관련성 높은 텍스트 청크 반환
    return [text_chunks[index] for index in top_indices]


## 추출된 청크에 대한 질의 실행 (이 부분은 이전 셀들과 중복되어 보입니다.)

In [21]:
# JSON 파일에서 검증 데이터 로드
with open('data/val.json') as f:
    data = json.load(f)

# 검증 데이터에서 첫 번째 질의 추출
query = data[0]['question']

# 질의에 대해 상위 2개의 가장 관련성 높은 텍스트 청크를 찾기 위해 시맨틱 검색 수행
top_chunks = semantic_search(query, text_chunks, response, k=2) # 여기서 response는 바로 위 셀에서 생성된 임베딩 객체

# 질의 출력
print("질의:", query)

# 상위 2개의 가장 관련성 높은 텍스트 청크 출력
for i, chunk in enumerate(top_chunks):
    print(f"문맥 {i + 1}:\n{chunk}\n=====================================")

Query: What is 'Explainable AI' and why is it considered important?
Context 1:
systems. Explainable AI (XAI) 
techniques aim to make AI decisions more understandable, enabling users to assess their 
fairness and accuracy. 
Privacy and Data Protection 
AI systems often rely on large amounts of data, raising concerns about privacy and data 
protection. Ensuring responsible data handling, implementing privacy-preserving techniques, 
and complying with data protection regulations are crucial. 
Accountability and Responsibility 
Establishing accountability and responsibility for AI systems is essential for addressing potential 
harms and ensuring ethical behavior. This includes defining roles and responsibilities for 
developers, deployers, and users of AI systems. 
Chapter 20: Building Trust in AI 
Transparency and Explainability 
Transparency and explainability are key to building trust in AI. Making AI systems understandable 
and providing insights into their decision-making processes he

## 검색된 청크를 기반으로 응답 생성 (이 부분은 이전 셀들과 중복되어 보입니다.)

In [22]:
# AI 어시스턴트에 대한 시스템 프롬프트 정의
system_prompt = "당신은 주어진 문맥에 기반하여 엄격하게 답변하는 AI 어시스턴트입니다. 제공된 문맥에서 직접적으로 답변을 도출할 수 없는 경우, '답변하기에 충분한 정보가 없습니다.'라고 응답하십시오."

def generate_response_from_chunks(system_prompt, user_message, model="meta-llama/Llama-3.2-3B-Instruct"):
    """ # 함수 이름을 generate_response_from_chunks로 변경하여 이전 generate_response 함수와 구분
    시스템 프롬프트와 사용자 메시지를 기반으로 AI 모델로부터 응답을 생성합니다.

    Args:
    system_prompt (str): AI의 행동을 안내하는 시스템 프롬프트.
    user_message (str): 사용자의 메시지 또는 질의.
    model (str): 응답 생성에 사용할 모델. 기본값은 "meta-llama/Llama-3.2-3B-Instruct"입니다.

    Returns:
    dict: AI 모델의 응답.
    """
    api_response = client.chat.completions.create( # 변수명을 response에서 api_response로 변경
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return api_response

# 상위 청크를 기반으로 사용자 프롬프트 생성
user_prompt_for_chunks = "\n".join([f"문맥 {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt_for_chunks = f"{user_prompt_for_chunks}\n질문: {query}"

# AI 응답 생성
ai_response_from_chunks = generate_response_from_chunks(system_prompt, user_prompt_for_chunks)


## AI 응답 평가 (이 부분은 이전 셀들과 중복되어 보입니다.)
AI 응답을 예상 답변과 비교하고 점수를 매깁니다.

In [23]:
# 평가 시스템에 대한 시스템 프롬프트 정의
evaluate_system_prompt_chunks = "당신은 AI 어시스턴트의 응답을 평가하는 지능형 평가 시스템입니다. AI 어시스턴트의 응답이 실제 응답과 매우 유사하면 1점을 부여하십시오. 응답이 실제 응답과 비교하여 부정확하거나 만족스럽지 않으면 0점을 부여하십시오. 응답이 실제 응답과 부분적으로 일치하면 0.5점을 부여하십시오."

# 사용자 질의, AI 응답, 실제 응답 및 평가 시스템 프롬프트를 결합하여 평가 프롬프트 생성
evaluation_prompt_chunks = f"사용자 질의: {query}\nAI 응답:\n{ai_response_from_chunks.choices[0].message.content}\n실제 응답: {data[0]['ideal_answer']}\n{evaluate_system_prompt_chunks}"

# 평가 시스템 프롬프트와 평가 프롬프트를 사용하여 평가 응답 생성
evaluation_response_chunks = generate_response_from_chunks(evaluate_system_prompt_chunks, evaluation_prompt_chunks)

# 평가 응답 출력
print(evaluation_response_chunks.choices[0].message.content)

Based on the evaluation criteria, I would assign a score of 0.8 to the AI assistant's response.

The response is very close to the true response, and it correctly conveys the main idea of Explainable AI (XAI) and its importance. The AI assistant has successfully identified the primary goal of XAI, its significance in building trust and accountability, and its relevance to areas such as privacy and data protection.

However, the response could be improved by providing more specific details and examples to support the claims made. For instance, the AI assistant could have elaborated on the techniques used in XAI, such as model interpretability, feature attribution, and explainability metrics. Additionally, the response could have provided more concrete examples of how XAI is being applied in various fields, such as healthcare and finance.

Overall, the response is a good start, but it could benefit from more depth and specificity to make it more accurate and informative.
